<a href="https://colab.research.google.com/github/Daivar/PP6_Adult_dataset/blob/main/Keras_adult_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import warnings 
warnings.filterwarnings('ignore')
import numpy as np
np.random.seed(7)

In [2]:
import pandas as pd

columns = ["Age", "WorkClass", "fnlwgt", "Education", "EducationNum",
        "MaritalStatus", "Occupation", "Relationship", "Race", "Gender",
        "CapitalGain", "CapitalLoss", "HoursPerWeek", "NativeCountry", "Income"]

data = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data',
                    header=None,
                    names=columns)

In [3]:
data.head()

,Age,WorkClass,fnlwgt,Education,EducationNum,MaritalStatus,Occupation,Relationship,Race,Gender,CapitalGain,CapitalLoss,HoursPerWeek,NativeCountry,Income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [4]:
data.describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
Age,32561,NaN,NaN,NaN,38.5816,13.6404,17,28,37,48,90
WorkClass,32561,9,Private,22696,NaN,NaN,NaN,NaN,NaN,NaN,NaN
fnlwgt,32561,NaN,NaN,NaN,189778,105550,12285,117827,178356,237051,1.48470e+06
Education,32561,16,HS-grad,10501,NaN,NaN,NaN,NaN,NaN,NaN,NaN
EducationNum,32561,NaN,NaN,NaN,10.0807,2.57272,1,9,10,12,16
MaritalStatus,32561,7,Married-civ-spouse,14976,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Occupation,32561,15,Prof-specialty,4140,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Relationship,32561,6,Husband,13193,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Race,32561,5,White,27816,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Gender,32561,2,Male,21790,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Age            32561 non-null  int64 
 1   WorkClass      32561 non-null  object
 2   fnlwgt         32561 non-null  int64 
 3   Education      32561 non-null  object
 4   EducationNum   32561 non-null  int64 
 5   MaritalStatus  32561 non-null  object
 6   Occupation     32561 non-null  object
 7   Relationship   32561 non-null  object
 8   Race           32561 non-null  object
 9   Gender         32561 non-null  object
 10  CapitalGain    32561 non-null  int64 
 11  CapitalLoss    32561 non-null  int64 
 12  HoursPerWeek   32561 non-null  int64 
 13  NativeCountry  32561 non-null  object
 14  Income         32561 non-null  object
dtypes: int64(6), object(9)
memory usage: 3.7+ MB


In [6]:
train = data[:30561]
test = data[30561:]

In [7]:
cat_feats = [column for column in data.columns if data[column].dtypes=='object']
non_cat_feats = [column for column in data.columns if data[column].dtypes!='object']

In [8]:
from sklearn.preprocessing import LabelEncoder

train[cat_feats] = train[cat_feats].apply(LabelEncoder().fit_transform)
test[cat_feats] = test[cat_feats].apply(LabelEncoder().fit_transform)

In [9]:
X_train = train.iloc[:,0:-1]
y_train = train['Income']

X_test = test.iloc[:,0:-1]
y_test = test['Income']

X_train.shape, y_train.shape, X_test.shape, y_test.shape

((30561, 14), (30561,), (2000, 14), (2000,))

In [10]:
X_train = train.iloc[:,0:-1]
y_train = train['Income']

X_test = test.iloc[:,0:-1]
y_test = test['Income']

X_train.shape, y_train.shape, X_test.shape, y_test.shape

((30561, 14), (30561,), (2000, 14), (2000,))

In [11]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

scaled_train = StandardScaler().fit_transform(X_train[non_cat_feats].values)
scaled_test = StandardScaler().fit_transform(X_test[non_cat_feats].values)

In [12]:
X_train[non_cat_feats] = scaled_train
X_test[non_cat_feats] = scaled_test

In [13]:
from tensorflow import keras

In [14]:
model = keras.models.Sequential()
model.add(keras.layers.Dropout(rate=0.2, input_shape=X_train.shape[1:]))
for _ in range(2):
        model.add(keras.layers.Dense(units=64, activation='relu'))
        model.add(keras.layers.Dropout(rate=0.2))
model.add(keras.layers.Dense(units=1, activation='sigmoid'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dropout (Dropout)           (None, 14)                0         
                                                                 
 dense (Dense)               (None, 64)                960       
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 64)                4160      
                                                                 
 dropout_2 (Dropout)         (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 1)                 65        
                                                                 
Total params: 5,185
Trainable params: 5,185
Non-trainabl

In [16]:
optimizer = keras.optimizers.Adam(lr=0.001)
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['acc'])
es_cb = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

In [18]:
import time

start = time.time()
history = model.fit(X_train, y_train,
         validation_data=(X_test, y_test),
         batch_size=64,
         epochs=100,
         callbacks=[es_cb])
print('\nIt took {} seconds'.format(time.time()-start))

Epoch 1/100
478/478 [==============================] - 2s 5ms/step - loss: 0.3633 - acc: 0.8299 - val_loss: 0.3532 - val_acc: 0.8330
Epoch 2/100
478/478 [==============================] - 1s 3ms/step - loss: 0.3643 - acc: 0.8289 - val_loss: 0.3497 - val_acc: 0.8300
Epoch 3/100
478/478 [==============================] - 2s 4ms/step - loss: 0.3651 - acc: 0.8298 - val_loss: 0.3630 - val_acc: 0.8400
Epoch 4/100
478/478 [==============================] - 1s 2ms/step - loss: 0.3649 - acc: 0.8288 - val_loss: 0.3575 - val_acc: 0.8305
Epoch 5/100
478/478 [==============================] - 1s 2ms/step - loss: 0.3644 - acc: 0.8287 - val_loss: 0.3479 - val_acc: 0.8340
Epoch 6/100
478/478 [==============================] - 1s 2ms/step - loss: 0.3652 - acc: 0.8280 - val_loss: 0.3538 - val_acc: 0.8330
Epoch 7/100
478/478 [==============================] - 1s 2ms/step - loss: 0.3618 - acc: 0.8305 - val_loss: 0.3442 - val_acc: 0.8340
Epoch 8/100
478/478 [==============================] - 1s 2ms/step - 